In [1]:
import pytorch3d.transforms as pt
import zarr
import numpy as np
import torch

In [2]:
def numpy_quaternion_to_6d(quaternions_np):
    # Convert the numpy array of quaternions to a PyTorch tensor
    quaternions = torch.from_numpy(quaternions_np).float()

    # Move the real part from the back to the front
    quaternions = torch.cat([quaternions[:, -1:], quaternions[:, :-1]], dim=1)

    # Convert each quaternion to a rotation matrix
    rot_mats = pt.quaternion_to_matrix(quaternions)

    # Extract the first two columns of each rotation matrix
    six_d_representations = pt.matrix_to_rotation_6d(rot_mats)

    # Convert the results back to a numpy array
    return six_d_representations.numpy()

In [3]:
dataset = zarr.open(
    "/data/scratch/ankile/furniture-data/processed/sim/feature/vip/combined.zarr",
    mode="a",
)

In [4]:
rot_quat = dataset["action"][:, 3:7]

rot_quat.shape

(1128470, 4)

In [5]:
rot_6d = numpy_quaternion_to_6d(rot_quat)

rot_6d.shape

(1128470, 6)

In [6]:
action_6d = np.concatenate(
    [dataset["action"][:, :3], rot_6d, dataset["action"][:, 7:]], axis=1
)

action_6d.shape

(1128470, 10)

In [7]:
dataset["action_6d"] = action_6d

In [2]:
# Get the stats and store them
from src.dataset.normalizer import get_data_stats

In [10]:
stats = get_data_stats(action_6d)

stats

{'min': array([-0.11999829, -0.11999908, -0.08132574, -0.35798502, -0.99999934,
        -0.70630395, -0.99999964, -0.28959465, -0.89785737, -1.        ],
       dtype=float32),
 'max': array([0.11999787, 0.11999953, 0.07999872, 1.        , 0.9999992 ,
        0.9999981 , 0.9999772 , 1.        , 0.49280044, 1.        ],
       dtype=float32)}

In [9]:
stats = get_data_stats(action_6d)

stats

{'min': array([-0.11999926, -0.1199997 , -0.08132574, -0.99995553, -0.9999999 ,
        -0.9951642 , -0.99999964, -0.99929166, -0.89785737, -1.        ],
       dtype=float32),
 'max': array([0.11999996, 0.11999957, 0.09052899, 1.        , 0.9999998 ,
        0.9999981 , 0.99999875, 1.        , 0.59187496, 1.        ],
       dtype=float32)}

In [4]:
dataset = zarr.open(
    "test.zarr",
    mode="w",
)

In [5]:
# Add nested groups to the root group
dataset.create_group("action")
dataset.create_group("action/6d")
dataset.create_group("action/quat")

<zarr.hierarchy.Group '/action/quat'>

In [6]:
# Add data to the nested groups
dataset["action/6d"] = np.random.rand(100, 6)
dataset["action/quat"] = np.random.rand(100, 4)

In [8]:
# Read the data in the 6D group
dataset["action"]["6d"][:]

array([[4.84917336e-01, 8.50956869e-01, 6.51772583e-01, 3.67672811e-02,
        6.48657343e-01, 2.48006422e-01],
       [4.68869580e-01, 5.87572016e-01, 9.15825188e-01, 8.92789343e-01,
        1.68138574e-01, 1.96886635e-01],
       [6.16899284e-01, 7.67222054e-01, 3.52560971e-01, 1.82418605e-02,
        7.23581650e-01, 6.93448497e-01],
       [6.34723881e-01, 5.25868809e-01, 2.89767354e-01, 3.77854309e-01,
        4.78354634e-01, 8.50765909e-01],
       [3.96843786e-01, 1.35549528e-02, 6.95884271e-01, 5.16376423e-01,
        2.99139903e-01, 1.60586352e-01],
       [4.34144796e-01, 2.33181798e-01, 5.46062537e-01, 1.90481390e-01,
        8.56172761e-01, 9.74144590e-01],
       [9.47299057e-02, 2.90233478e-01, 6.00643081e-01, 8.10123522e-01,
        7.90540294e-01, 3.59834045e-01],
       [5.09921135e-01, 2.89706290e-01, 5.72463171e-02, 3.43007171e-01,
        6.56968534e-01, 8.25113748e-01],
       [6.09609445e-03, 1.78463010e-01, 9.71690445e-01, 7.45251627e-01,
        9.01785251e-01, 

In [ ]:
from glob import glob
from pathlib import Path
from src.visualization.render_mp4 import unpickle_data

In [13]:
paths = [
    Path(p)
    for p in glob(
        "/data/scratch-oc40/pulkitag/ankile/furniture-data/raw/sim/one_leg/scripted/**/success/*.pkl*",
        recursive=True,
    )
]

In [15]:
data = unpickle_data(paths[0])

In [17]:
len(data["observations"]), len(data["actions"]), len(data["rewards"])

(576, 575, 575)